In [6]:
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms
from detection_utils import load_yolo_model, draw_bounding_boxes

# Check if GPU is available and use it, otherwise fallback to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the DehazeNet model architecture
class DehazeNet(torch.nn.Module):
    def __init__(self):
        super(DehazeNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 3, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

# Load the DehazeNet model
model_path = "D:\\Mini project Git\\Dehazing-and-Human-Detection-Project\\Dehazing_model\\dehaze_model.pth"
model = DehazeNet().to(device)
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("DehazeNet model loaded successfully.")
except RuntimeError as e:
    print(f"Error loading the DehazeNet model: {e}")
model.eval()

# Define the image transformation (same as used during training)
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Reverse the normalization after prediction
def reverse_normalization(tensor):
    return tensor * 0.5 + 0.5

# Function to process frames through DehazeNet
def process_frame(frame):
    input_frame = transform(frame).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_frame).cpu()
    output_frame = output.squeeze(0).permute(1, 2, 0).numpy()
    output_frame = reverse_normalization(output_frame)
    output_frame = np.clip(output_frame * 255, 0, 255).astype(np.uint8)
    return output_frame

# Load YOLO model
config_path = "D:\\Mini project Git\\Dehazing-and-Human-Detection-Project\\Human_Detection\\yolov3.cfg"
weights_path = "D:\\Mini project Git\\Dehazing-and-Human-Detection-Project\\Human_Detection\\yolov3.weights"
labels = ["person", "cat", "dog"]
net, output_layers = load_yolo_model(config_path, weights_path)




DehazeNet model loaded successfully.


In [9]:
# Video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert BGR to RGB for processing
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Dehaze the frame
    dehazed_frame = process_frame(frame_rgb)

    # YOLO object detection
    blob = cv2.dnn.blobFromImage(dehazed_frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id < len(labels) and labels[class_id] in ["person", "cat", "dog"]:
                center_x, center_y = int(detection[0] * frame.shape[1]), int(detection[1] * frame.shape[0])
                w, h = int(detection[2] * frame.shape[1]), int(detection[3] * frame.shape[0])
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply Non-Maximum Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indices) > 0:
        indices = indices.flatten()
        boxes = [boxes[i] for i in indices]
        confidences = [confidences[i] for i in indices]
        class_ids = [class_ids[i] for i in indices]

    # Count humans and animals
    human_count = sum(1 for id in class_ids if labels[id] == "person")
    animal_count = sum(1 for id in class_ids if labels[id] in ["cat", "dog"])

    # Convert dehazed frame back to BGR for OpenCV
    dehazed_frame_bgr = cv2.cvtColor(dehazed_frame, cv2.COLOR_RGB2BGR)

    # Draw bounding boxes
    annotated_frame = draw_bounding_boxes(dehazed_frame_bgr, boxes, confidences, class_ids, labels)

    # Display counts
    cv2.putText(annotated_frame, f"Humans: {human_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(annotated_frame, f"Animals: {animal_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show the annotated frame
    cv2.imshow("Dehazed Frame with Detection", annotated_frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()